In [ ]:
!pip install cryptography

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hmac
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# Constants for AES encryption
KEY_SIZE = 32  # 256 bits for AES
IV_SIZE = 16   # 128 bits for AES IV
SALT_SIZE = 16 # Size of the salt for key derivation
PAD_BLOCK_SIZE = 128

# Function to generate a key from a password
def generate_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=KEY_SIZE,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

# Function to encrypt a file
def encrypt_file(file_path: str, password: str, output_path: str):
    # Generate salt and key
    salt = os.urandom(SALT_SIZE)
    key = generate_key(password, salt)
    iv = os.urandom(IV_SIZE)

    # Read file data
    with open("/content/example.txt", 'rb') as file:
        data = file.read()

    # Pad data
    padder = padding.PKCS7(PAD_BLOCK_SIZE).padder()
    padded_data = padder.update(data) + padder.finalize()

    # Encrypt data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Write encrypted data to file
    with open(output_path, 'wb') as enc_file:
        enc_file.write(salt + iv + encrypted_data)

    print(f"File encrypted and saved to {output_path}")

# Function to decrypt a file
def decrypt_file(file_path: str, password: str, output_path: str):
    # Read the salt, IV, and encrypted data
    with open(file_path, 'rb') as enc_file:
        salt = enc_file.read(SALT_SIZE)
        iv = enc_file.read(IV_SIZE)
        encrypted_data = enc_file.read()

    # Generate key from the password and salt
    key = generate_key(password, salt)

    # Decrypt data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad data
    unpadder = padding.PKCS7(PAD_BLOCK_SIZE).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()

    # Write decrypted data to file
    with open(output_path, 'wb') as dec_file:
        dec_file.write(data)

    print(f"File decrypted and saved to {output_path}")

# Usage example
if __name__ == "__main__":
    password = "strong_password"
    input_file = "example.txt"           # Replace with the path to your file
    encrypted_file = "example.enc"        # Output path for encrypted file
    decrypted_file = "example_decrypted.txt"  # Output path for decrypted file

    # Encrypt the file
    encrypt_file(input_file, password, encrypted_file)

    # Decrypt the file
    decrypt_file(encrypted_file, password, decrypted_file)


File encrypted and saved to example.enc
File decrypted and saved to example_decrypted.txt
